In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-mwe3cn1u/kobart_4667564dd7714d45bdd55a1bbd6752a0
  Running command git clone --filter=blob:none --quiet https://github.com/SKT-AI/KoBART /tmp/pip-install-mwe3cn1u/kobart_4667564dd7714d45bdd55a1bbd6752a0
  Resolved https://github.com/SKT-AI/KoBART to commit 30c5eb7b593828d6ec2d767eeedb2f2ed02c5c2a
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.26.120-py3-none-any.whl (135 kB)
  Using cached pytorch_lightning-1.2.1-py3-none-any.whl (814 kB)
INFO: pip is looking at multiple versions of kobart to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from kobart) (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0)
ERROR: No matching distribution found for torch==1.7.1


In [5]:
!pip install torchtext==0.8.0

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement torchtext==0.8.0 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1)
ERROR: No matching distribution found for torchtext==0.8.0


In [6]:
!pip install pyyaml==5.4.1

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import pandas as pd
import numpy as np
from transformers import TrainingArguments, Trainer
from transformers.models.bart.modeling_bart import shift_tokens_right
from datasets import Dataset
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import torch

from datasets import load_dataset
import os

def file_exists(filename):
    return os.path.isfile(filename)


def load_dataset(dataset_name, column_names):
    loaded_features = {}
    for feature in column_names:
        # 문자열 특성이 있는 경우 별도로 처리
        if feature in ["source", "target", "Id"]:
            filename = f"{dataset_name}_{feature}.txt"
            if file_exists(filename):
                with open(filename, "r") as f:
                    loaded_features[feature] = [line.strip() for line in f.readlines()]
            else:
                print(f"Warning: {filename} not found.")
        else:
            filename = f"{dataset_name}_{feature}.pt"
            if file_exists(filename):
                feature_tensor = torch.load(filename)
                loaded_features[feature] = feature_tensor.tolist()  # 텐서를 리스트로 변환
            else:
                print(f"Warning: {filename} not found.")

    min_len = min([len(v) for v in loaded_features.values()])  # 모든 특성의 최소 길이를 구합니다.
    for key in loaded_features:
        loaded_features[key] = loaded_features[key][:min_len]  # 모든 특성의 길이를 최소 길이로 잘라줍니다.

    return Dataset.from_dict(loaded_features)

# train_dataset과 test_dataset의 열(column) 이름을 명시해야 합니다.
column_names = ['Id', 'source', 'target', 'input_ids', 'attention_mask', 'decoder_input_ids', 'labels']

# train_dataset과 test_dataset 로드
train_dataset = load_dataset("train_dataset", column_names)
test_dataset = load_dataset("test_dataset", column_names)


2023-04-26 09:06:43.011115: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-26 09:06:43.058711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 09:06:44.473234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
train_dataset

Dataset({
    features: ['Id', 'source', 'target', 'input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
    num_rows: 146771
})

In [9]:
test_dataset

Dataset({
    features: ['Id', 'source', 'target', 'input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
    num_rows: 18300
})

In [10]:
# BART 토크나이저 및 모델 불러오기
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [11]:
# GPU 사용 가능한지 확인하고 기본 장치로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# GPU 설정
if torch.cuda.is_available():
    print("GPU is available!")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead.")
    device = torch.device("cpu")


GPU is available!


In [13]:
# 모델을 GPU로 이동
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [14]:
# 변환된 데이터셋의 형식 지정
columns = ['input_ids', 'labels', 'decoder_input_ids', 'attention_mask']
train_dataset.set_format(type='torch', columns=columns)
test_dataset.set_format(type='torch', columns=columns)

In [15]:
train_dataset

Dataset({
    features: ['Id', 'source', 'target', 'input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
    num_rows: 146771
})

In [16]:
test_dataset

Dataset({
    features: ['Id', 'source', 'target', 'input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
    num_rows: 18300
})

In [17]:
# 훈련 인수 설정
training_args = TrainingArguments(
    output_dir='./models/bart-summarizer',          
    num_train_epochs=1,           
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,   
    warmup_steps=500,               
    weight_decay=0.01,              
    logging_dir='./logs',          
    fp16=True,  # 추가: GPU 사용 시 mixed precision 학습 활성화
    save_steps=1000,  # 체크포인트 저장 간격을 변경합니다. 이 값은 원하는 숫자로 조절할 수 있습니다.
    save_total_limit=1,# 저장할 체크포인트 수를 제한합니다. 이 값은 원하는 숫자로 조절할 수 있습니다.
    load_best_model_at_end=True,
    evaluation_strategy='steps',  # 추가: 평가 전략을 'steps'로 설정합니다.
    save_strategy='steps',  # 추가: 저장 전략을 'steps'로 설정합니다.
    eval_steps=1000, # 추가: 평가 간격을 설정합니다. 이 값은 원하는 숫자로 조절할 수 있습니다.
    gradient_accumulation_steps=1,
)

In [18]:
!pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable


In [19]:
from rouge_score import rouge_scorer
from datasets import load_metric

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge 점수 계산을 위한 Scorer 생성
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    metric = load_metric("rouge")

    # 예측과 라벨 간 Rouge 점수 계산
    scores = []
    for pred, label in zip(decoded_preds, decoded_labels):
        scores.append(scorer.score(pred, label))

    # 평균 Rouge 점수 계산
    final_scores = {key: 0 for key in ['rouge1', 'rouge2', 'rougeL']}
    for score in scores:
        for key in final_scores.keys():
            final_scores[key] += score[key].fmeasure
    final_scores = {key: value / len(scores) for key, value in final_scores.items()}

    return final_scores


In [20]:
# 트레이너 인스턴스 생성
trainer = Trainer(
    model=model,                       
    args=training_args,                  
    train_dataset=train_dataset,        
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [21]:
# 모델 훈련 시작
trainer.train()

/home/elicer/.local/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.15 GiB (GPU 0; 9.50 GiB total capacity; 4.64 GiB already allocated; 3.04 GiB free; 5.45 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 모델 평가
yhat = trainer.predict(test_dataset)
print(yhat)

In [ ]:
{'eval_loss': 0.4558240473270416, 'eval_runtime': 1799.2075, 'eval_samples_per_second': 10.171, 'eval_steps_per_second': 10.171}

In [34]:
torch.cuda.empty_cache()

In [21]:
!nvidia-smi

Wed Apr 26 08:31:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:65:00.0 Off |                   On |
| N/A   30C    P0    60W / 300W |                  N/A |     N/A      Default |
|                               |                      |              Enabled |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| MIG d

In [24]:
!df -h

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   11G   90G  11% /
/dev/sda4       860G   30G  830G   4% /rt
tmpfs            64M     0   64M   0% /dev
tmpfs           101G  5.5M  101G   1% /etc/hosts
shm             4.0G  4.0K  4.0G   1% /dev/shm
/dev/loop0      100G   11G   90G  11% /mnt/elice/shared
tmpfs           504G   12K  504G   1% /proc/driver/nvidia
overlay         100G   11G   90G  11% /usr/lib/x86_64-linux-gnu/libcuda.so.520.61.05
